# Objetivo:

- Recuperação dos dados da Camada Bronze;
- Filtro de colunas;
- Tipagem dos Dados;
- Sanitização da Base;
- Salvar saída das tabelas na camada Silver.

### Import das Bibliotecas

In [1]:
import pandas as pd
import os

### Definição dos Diretórios

In [2]:
DATA_DIR = '../data'

BRONZE_DIR = os.path.join(DATA_DIR, 'bronze')
SILVER_DIR = os.path.join(DATA_DIR, 'silver')

# Garante que o diretório silver exista

os.makedirs(SILVER_DIR, exist_ok=True)

### Definição dos nomes dos Arquivos na camada Bronze Dinâmico

In [3]:
EMPRESA_FILE_BRONZE = None
SOCIO_FILE_BRONZE = None

# Lista todos os arquivos no diretório bronze

try:
    bronze_files = os.listdir(BRONZE_DIR)

    # Procura pelos arquivos de Empresa e Socio com base no sufixo
    for filename in bronze_files:
        if filename.endswith('.EMPRECSV'):
            EMPRESA_FILE_BRONZE = os.path.join(BRONZE_DIR, filename)
        elif filename.endswith('.SOCIOCSV'):
            SOCIO_FILE_BRONZE = os.path.join(BRONZE_DIR, filename)

    # Verifica se os arquivos foram encontrados
    if EMPRESA_FILE_BRONZE is None:
        raise FileNotFoundError(f"Nenhum arquivo terminando com '.EMPRECSV' encontrado em {BRONZE_DIR}. Verifique o nome do arquivo.")
    if SOCIO_FILE_BRONZE is None:
        raise FileNotFoundError(f"Nenhum arquivo terminando com '.SOCIOCSV' encontrado em {BRONZE_DIR}. Verifique o nome do arquivo.")

except FileNotFoundError as e:
    print(f"ERRO: {e}")
    exit()
except Exception as e:
    print(f"Ocorreu um erro ao listar arquivos no diretório bronze: {e}")
    exit()

#### Nome dos Arquivos na Camada Silver

In [4]:
# Nomes dos arquivos na camada silver

EMPRESA_FILE_SILVER = os.path.join(SILVER_DIR, 'empresas_silver.parquet')
SOCIO_FILE_SILVER = os.path.join(SILVER_DIR, 'socios_silver.parquet')

### Parâmetros de Leitura dos Arquivos

In [5]:
# --- Parâmetros de Leitura dos Arquivos (Comum para Receita Federal) ---
# Os arquivos da Receita Federal geralmente são delimitados por ponto e vírgula,
# codificados em latin-1 e não possuem cabeçalho.
READ_PARAMS = {
    'sep': ';',
    'encoding': 'latin-1',
    'header': None,
    'dtype': str # Ler tudo como string inicialmente para evitar erros de tipo
}

### Tabela Empresas

#### Leitura da Raw

In [6]:
# Ler tabela Empresas

df_empresas_raw = pd.read_csv(EMPRESA_FILE_BRONZE, **READ_PARAMS)
display(df_empresas_raw)

,0,1,2,3,4,5,6
0,41273590,MARIA DAS MERCES SOARES LEMOS,4014,34,"0,00",05,NaN
1,41273591,CASSIO APARECIDO LOPES 06457523803,2135,50,"1000,00",01,NaN
2,41273592,41.273.592 HELIO DE JESUS PEREIRA,2135,50,"30000,00",01,NaN
3,41273593,JULIO CESAR NUNES 39611300867,2135,50,"3000,00",01,NaN
4,41273594,OZINETE DELFINO CALDAS 41608224287,2135,50,"5000,00",01,NaN
...,...,...,...,...,...,...,...
23537411,98819550,ANTONIO ORIQUES CARDOSO,2135,50,"0,00",01,NaN
23537412,98819568,ELI ORIQUES CARDOSO,2135,50,"0,00",01,NaN
23537413,98819600,PAULINO LEMOS DA SILVA,2135,50,"0,00",05,NaN
23537414,98819832,JOAO FELTRIN,2135,50,"0,00",01,NaN


#### Camada Silver

In [7]:
# Seleciona e renomeia as colunas

df_empresas_silver = df_empresas_raw[[0, 1, 2, 3, 4, 5]].copy()
df_empresas_silver.columns = [
    'cnpj',
    'razao_social',
    'natureza_juridica',
    'qualificacao_responsavel',
    'capital_social',
    'cod_porte'
]

In [8]:
# Conversão de tipos de dados
# Usar pd.to_numeric com errors='coerce' para tratar valores não numéricos como NaN
# e depois preencher NaN com um valor padrão ou remover se necessário

df_empresas_silver['natureza_juridica'] = pd.to_numeric(df_empresas_silver['natureza_juridica'], errors='coerce').fillna(-1).astype(int)
df_empresas_silver['qualificacao_responsavel'] = pd.to_numeric(df_empresas_silver['qualificacao_responsavel'], errors='coerce').fillna(-1).astype(int)
df_empresas_silver['capital_social'] = pd.to_numeric(df_empresas_silver['capital_social'].str.replace(',', '.'), errors='coerce').fillna(0.0).astype(float)
df_empresas_silver = df_empresas_silver.astype({"cnpj": "string", "razao_social": "string", "cod_porte": "string"})

# Limpeza de strings: remover espaços em branco extras

df_empresas_silver['razao_social'] = df_empresas_silver['razao_social'].str.strip()
df_empresas_silver['cod_porte'] = df_empresas_silver['cod_porte'].str.strip()

In [9]:
print(f"Tabela de Empresas processada. Total de registros: {len(df_empresas_silver)}")
print("Primeiras 5 linhas da tabela Empresas (Silver):")
print(df_empresas_silver.head())
print("\nTipos de dados da tabela Empresas (Silver):")
print(df_empresas_silver.dtypes)

Tabela de Empresas processada. Total de registros: 23537416
Primeiras 5 linhas da tabela Empresas (Silver):
       cnpj                        razao_social  natureza_juridica  \
0  41273590       MARIA DAS MERCES SOARES LEMOS               4014   
1  41273591  CASSIO APARECIDO LOPES 06457523803               2135   
2  41273592   41.273.592 HELIO DE JESUS PEREIRA               2135   
3  41273593       JULIO CESAR NUNES 39611300867               2135   
4  41273594  OZINETE DELFINO CALDAS 41608224287               2135   

   qualificacao_responsavel  capital_social cod_porte  
0                        34             0.0        05  
1                        50          1000.0        01  
2                        50         30000.0        01  
3                        50          3000.0        01  
4                        50          5000.0        01  

Tipos de dados da tabela Empresas (Silver):
cnpj                        string[python]
razao_social                string[python]
natu

In [10]:
# Salva o DataFrame Empresa na camada Silver

df_empresas_silver.to_parquet(EMPRESA_FILE_SILVER, index=False)
print(f"Tabela de Empresas salva em: {EMPRESA_FILE_SILVER}")

Tabela de Empresas salva em: ../data/silver/empresas_silver.parquet


### Tabela Sócios

#### Leitura Raw

In [6]:
# Ler tabela Sócios

df_socios_raw = pd.read_csv(SOCIO_FILE_BRONZE, **READ_PARAMS)
display(df_socios_raw)

,0,1,2,3,4,5,6,7,8,9,10
0,55499544,2,PAOLA ZORZI GARRINI,***324158**,49,20240612,NaN,***000000**,NaN,00,4
1,55470492,2,MAURI TESKE XAVIER DE BARROS,***856669**,49,20240610,NaN,***000000**,NaN,00,5
2,55470492,2,ARTUR TESKE XAVIER DE BARROS,***217579**,49,20240610,NaN,***000000**,NaN,00,3
3,55520028,2,ESTHER CHAGAS GLASS,***347028**,49,20240613,NaN,***000000**,NaN,00,4
4,55289082,2,GUILHERME FERREIRA CARDIAS,***126540**,65,20240527,NaN,***000000**,NaN,00,4
...,...,...,...,...,...,...,...,...,...,...,...
8024947,62150102,2,MARIA STELLA NUNES ARAUJO MOREIRA,***007968**,65,20250724,NaN,***000000**,NaN,00,6
8024948,62150102,2,FLAVIA GONCALVES,***842428**,05,20250724,NaN,***000000**,NaN,00,5
8024949,62150668,2,IAN OLIVEIRA CALACA DA COSTA,***487141**,49,20250809,NaN,***000000**,NaN,00,3
8024950,62150703,2,MARIA RAYANNE DE OLIVEIRA GUIMARAES,***475983**,49,20250809,NaN,***000000**,NaN,00,3


#### Camada Silver

In [7]:
# Seleciona e renomeia as colunas

df_socios_silver = df_socios_raw[[0, 1, 2, 3, 4]].copy()
df_socios_silver.columns = [
    'cnpj',
    'tipo_socio',
    'nome_socio',
    'documento_socio',
    'codigo_qualificacao_socio'
]

In [8]:
# Conversão de tipos de dados

df_socios_silver['tipo_socio'] = pd.to_numeric(df_socios_silver['tipo_socio'], errors='coerce').fillna(-1).astype(int)
df_socios_silver = df_socios_silver.astype({"cnpj": "string", "nome_socio": "string", "documento_socio": "string", "codigo_qualificacao_socio": "string"})

# Limpeza de strings: remover espaços em branco extras

df_socios_silver['nome_socio'] = df_socios_silver['nome_socio'].str.strip()
df_socios_silver['documento_socio'] = df_socios_silver['documento_socio'].str.strip()
df_socios_silver['codigo_qualificacao_socio'] = df_socios_silver['codigo_qualificacao_socio'].str.strip()

In [9]:
print(f"Tabela de Sócios processada. Total de registros: {len(df_socios_silver)}")
print("Primeiras 5 linhas da tabela Sócios (Silver):")
print(df_socios_silver.head())
print("\nTipos de dados da tabela Sócios (Silver):")
print(df_socios_silver.dtypes)

Tabela de Sócios processada. Total de registros: 8024952
Primeiras 5 linhas da tabela Sócios (Silver):
       cnpj  tipo_socio                    nome_socio documento_socio  \
0  55499544           2           PAOLA ZORZI GARRINI     ***324158**   
1  55470492           2  MAURI TESKE XAVIER DE BARROS     ***856669**   
2  55470492           2  ARTUR TESKE XAVIER DE BARROS     ***217579**   
3  55520028           2           ESTHER CHAGAS GLASS     ***347028**   
4  55289082           2    GUILHERME FERREIRA CARDIAS     ***126540**   

  codigo_qualificacao_socio  
0                        49  
1                        49  
2                        49  
3                        49  
4                        65  

Tipos de dados da tabela Sócios (Silver):
cnpj                         string[python]
tipo_socio                            int64
nome_socio                   string[python]
documento_socio              string[python]
codigo_qualificacao_socio    string[python]
dtype: object


In [10]:
# Salva o DataFrame Sócio na camada Silver

df_socios_silver.to_parquet(SOCIO_FILE_SILVER, index=False)
print(f"Tabela de Sócios salva em: {SOCIO_FILE_SILVER}")

Tabela de Sócios salva em: ../data/silver/socios_silver.parquet
